## March 19th

### We will look at health condition variables against initial waves using the aggregate health variable in indresp. 

In [2]:
import pandas as pd
import numpy as np

In [ ]:
data = {"state": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
        "year": [2000, 2001, 2002, 2001, 2002, 2003],
        "pop": [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

frame.tail()

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])

data


In [7]:
# If you add dataframe with no common rows or columns, it'll display null

df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"B": [3,4]})

df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


In [8]:
# Without the fill_value, it'll display NaN. 
df1.add(df2, fill_value=0)

# result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])


,A,B
0,1.0,3.0
1,2.0,4.0


In [14]:
obj = pd.Series(np.arange(6), index=["11220", "11220", "11340", "10001", "11190", "11000"])

obj

obj.sort_index()

10001    3
11000    5
11190    4
11220    0
11220    1
11340    2
dtype: int64

In [15]:
# Limits the display to 10 rows and columns
pd.options.display.max_rows = 10 

df = pd.read_stata("/Users/gavinqu/Desktop/School/Dissertation/UKDA-6614-stata/stata/stata13_se/ukhls/a_indresp.dta")

df

,pidp,a_hidp,a_pno,a_hhorig,a_memorig,a_psu,a_strata,a_sampst,a_month,a_ivfio,...,a_paynu_if,a_seearngrs_if,a_fiyrinvinc_if,a_fibenothr_if,a_fimnlabgrs_if,a_fimngrs_if,a_indpxus_xw,a_indinus_xw,a_indscus_xw,a_ind5mus_xw
0,68001367,68001363,1,ukhls gb 2009-10,ukhls gb 2009-10,2012,2006,osm,jan yr1,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.759427,0.727945,0.761283,0.000000
1,68004087,68004083,1,ukhls gb 2009-10,ukhls gb 2009-10,2012,2006,osm,jan yr1,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.905079,0.866007,0.846114,0.000000
2,68006127,68006123,1,ukhls gb 2009-10,ukhls gb 2009-10,2012,2006,osm,jan yr1,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.855818,0.814239,0.824023,0.000000
3,68006135,68006123,3,ukhls gb 2009-10,ukhls gb 2009-10,2012,2006,osm,jan yr1,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.971394,0.962062,0.972999,0.000000
4,68006807,68006803,1,ukhls gb 2009-10,ukhls gb 2009-10,2012,2006,osm,jan yr1,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.836741,0.793299,0.768091,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50989,1634643847,1634643843,1,ukhls emboost 2009-10,ukhls emboost 2009-10,51775,5117,osm,dec yr2,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.140098,0.142939,0.000000,0.013447
50990,1634644527,1634644523,1,ukhls emboost 2009-10,ukhls emboost 2009-10,51776,5117,osm,dec yr2,full interview,...,not imputed,imputed,never imputed,0.0,1.0,1.00,0.154505,0.171093,0.274597,0.021537
50991,1634644531,1634644523,2,ukhls emboost 2009-10,ukhls emboost 2009-10,51776,5117,osm,dec yr2,full interview,...,not imputed,never imputed,never imputed,0.07,0.0,0.07,0.141862,0.147306,0.212138,0.014134
50992,1634649967,1634649963,1,ukhls emboost 2009-10,ukhls emboost 2009-10,51784,5117,osm,dec yr2,full interview,...,not imputed,never imputed,never imputed,0.0,0.0,0.00,0.123934,0.115670,0.000000,0.015361


In [18]:
# Dummy Variable
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})

pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [19]:
np.random.seed(12345)

values = np.random.uniform(size=10)

bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


In [20]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

# pandas.get_dummies function converts this one-dimensional categorical data into a DataFrame 
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0
